In [1]:
import numpy as np
import torch
import sys
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm
import json
import atexit
import os
from typing import List, Dict, Tuple, Any
import colorama
from colorama import Fore, Style
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2ForSequenceClassification
import librosa
import pandas as pd
import warnings
import contextlib

# Initialize colorama
colorama.init()

def cleanup_dataloader():
    torch.utils.data._utils.worker._worker_info = None

# Register cleanup function
atexit.register(cleanup_dataloader)

class SuppressOutput:
    """Context manager to suppress stdout and stderr"""
    def __init__(self):
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        self.save_fds = [os.dup(1), os.dup(2)]

    def __enter__(self):
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)
        warnings.simplefilter("ignore")

    def __exit__(self, *_):
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        for fd in self.null_fds + self.save_fds:
            os.close(fd)
        warnings.resetwarnings()

class ModelLoader:
    """Handles loading of pretrained models"""
    @staticmethod
    def load_wav2vec2(model_path: str) -> Tuple[nn.Module, Any]:
        """Load Wav2Vec2 model and feature extractor"""
        try:
            # Load model
            model = Wav2Vec2ForSequenceClassification.from_pretrained(model_path)
            feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_path)
            
            # Load label map
            with open(os.path.join(model_path, "label_map.json"), "r") as f:
                label_map = json.load(f)
            
            return model, feature_extractor, label_map
        except Exception as e:
            raise Exception(f"Error loading Wav2Vec2 model: {str(e)}")

    @staticmethod
    def load_yamnet(model_path: str) -> nn.Module:
        """Load YAMNet model"""
        try:
            from yamnet import YAMNetBase  # Import from previous script
            
            # Load label map
            with open(os.path.join(model_path, "label_map.json"), "r") as f:
                label_map = json.load(f)
            
            # Initialize model
            model = YAMNetBase(num_classes=len(label_map))
            
            # Load state dict
            model.load_state_dict(torch.load(os.path.join(model_path, "model.pt")))
            
            return model, label_map
        except Exception as e:
            raise Exception(f"Error loading YAMNet model: {str(e)}")

class Visualizer:
    """Enhanced visualization capabilities"""
    def __init__(self, label_map: Dict[str, int]):
        self.label_map = label_map
        self.rev_label_map = {v: k for k, v in label_map.items()}
        
        # Set default matplotlib style
        plt.style.use('default')
        # Apply seaborn styling
        sns.set_theme(style="whitegrid")

    def plot_confusion_matrix(
        self,
        cm: np.ndarray,
        method_name: str,
        output_dir: str
    ) -> None:
        """Plot confusion matrix heatmap"""
        plt.figure(figsize=(10, 8))
        sns.heatmap(
            cm,
            annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=self.label_map.keys(),
            yticklabels=self.label_map.keys()
        )
        plt.title(f'Confusion Matrix - {method_name}')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()  # Added to prevent label cutoff
        
        # Save plot
        plt.savefig(os.path.join(output_dir, f'confusion_matrix_{method_name}.png'), 
                    bbox_inches='tight')  # Added to ensure full save
        plt.close()

    def plot_roc_curves(
        self,
        probabilities: Dict[str, np.ndarray],
        true_labels: np.ndarray,
        output_dir: str
    ) -> None:
        """Plot ROC curves for all methods"""
        plt.figure(figsize=(12, 8))
        
        # Use a color palette for consistent colors
        colors = sns.color_palette("husl", n_colors=len(probabilities)*len(self.label_map))
        color_idx = 0
        
        for method_name, probs in probabilities.items():
            for i in range(len(self.label_map)):
                # Prepare binary labels
                binary_labels = (true_labels == i).astype(int)
                
                # Calculate ROC curve
                if method_name in ['majority_voting', 'meta_classifier']:
                    # For discrete predictions, create pseudo-probabilities
                    class_probs = (probs == i).astype(float)
                else:
                    class_probs = probs[:, i]
                
                try:
                    fpr, tpr, _ = roc_curve(binary_labels, class_probs)
                    roc_auc = auc(fpr, tpr)
                    
                    # Plot ROC curve with consistent color
                    plt.plot(
                        fpr,
                        tpr,
                        color=colors[color_idx],
                        label=f'{method_name} - {self.rev_label_map[i]} (AUC = {roc_auc:.2f})'
                    )
                    color_idx += 1
                except ValueError as e:
                    print(f"Warning: Could not compute ROC curve for {method_name} - {self.rev_label_map[i]}: {str(e)}")
        
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves for All Methods')
        plt.legend(loc="center left", bbox_to_anchor=(1, 0.5))
        plt.tight_layout()
        
        # Save plot
        plt.savefig(os.path.join(output_dir, 'roc_curves.png'),
                    bbox_inches='tight',
                    dpi=300)  # Increased DPI for better quality
        plt.close()

    def plot_performance_comparison(
        self,
        results: Dict[str, Dict[str, float]],
        output_dir: str
    ) -> None:
        """Plot performance metrics comparison"""
        metrics = ['accuracy', 'precision', 'recall', 'f1']
    
        # Filter out the 'best_method' entry since it has a different structure
        methods = [method for method in results.keys() if method != 'best_method']
    
        # Prepare data for plotting
        data = []
        for method in methods:
            for metric in metrics:
                data.append({
                    'Method': method,
                    'Metric': metric,
                    'Score': results[method][metric]
                })
    
        df = pd.DataFrame(data)
    
        # Create grouped bar plot
        plt.figure(figsize=(12, 6))
        g = sns.barplot(
            x='Method',
            y='Score',
            hue='Metric',
            data=df,
            palette='husl'
        )
    
        plt.title('Performance Comparison Across Methods')
        plt.xticks(rotation=45, ha='right')  # Improved readability
    
        # Add value labels on the bars
        for container in g.containers:
            g.bar_label(container, fmt='%.2f', padding=3)
    
        plt.tight_layout()
    
        # Save plot
        plt.savefig(os.path.join(output_dir, 'performance_comparison.png'),
                    bbox_inches='tight',
                    dpi=300)
        plt.close()

    def plot_prediction_distribution(
        self,
        predictions: Dict[str, np.ndarray],
        output_dir: str
    ) -> None:
        """Plot distribution of predictions for each method"""
        plt.figure(figsize=(12, 6))
        
        x = np.arange(len(self.label_map))
        width = 0.8 / len(predictions)
        
        # Use a consistent color palette
        colors = sns.color_palette("husl", n_colors=len(predictions))
        
        for i, (method, preds) in enumerate(predictions.items()):
            counts = np.bincount(preds, minlength=len(self.label_map))
            plt.bar(
                x + i * width,
                counts,
                width,
                label=method,
                color=colors[i],
                alpha=0.8
            )
        
        plt.xlabel('Classes')
        plt.ylabel('Number of Predictions')
        plt.title('Distribution of Predictions Across Methods')
        plt.xticks(x + width * len(predictions) / 2, 
                   self.label_map.keys(),
                   rotation=45,
                   ha='right')
        plt.legend(loc='upper right')
        plt.tight_layout()
        
        # Save plot
        plt.savefig(os.path.join(output_dir, 'prediction_distribution.png'),
                    bbox_inches='tight',
                    dpi=300)
        plt.close()

class EnsembleClassifier:
    """Enhanced Ensemble Classifier with prediction capabilities"""
    def __init__(
        self,
        wav2vec_path: str,
        yamnet_path: str,
        device: torch.device = None
    ):
        self.device = device or torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Load models
        print("Loading Wav2Vec2 model...")
        self.wav2vec_model, self.feature_extractor, self.label_map = ModelLoader.load_wav2vec2(wav2vec_path)
        self.wav2vec_model.to(self.device)
        self.wav2vec_model.eval()  # Set to evaluation mode
        
        print("Loading YAMNet model...")
        self.yamnet_model, yamnet_label_map = ModelLoader.load_yamnet(yamnet_path)
        self.yamnet_model.to(self.device)
        self.yamnet_model.eval()  # Set to evaluation mode
        
        # Initialize other components
        self.meta_classifier = None
        self.visualizer = Visualizer(self.label_map)

    def get_model_predictions(
        self,
        data_loader: torch.utils.data.DataLoader,
        model_type: str
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Get predictions from specified model
        
        Args:
            data_loader: DataLoader containing test data
            model_type: Either 'wav2vec' or 'yamnet'
            
        Returns:
            Tuple of (predictions probabilities, true labels)
        """
        all_probs = []
        all_labels = []
        
        with torch.no_grad():
            for batch in tqdm(data_loader, desc=f"Getting {model_type} predictions"):
                inputs, labels = batch
                
                if model_type == 'wav2vec':
                    # Convert batch tensor to list of numpy arrays
                    input_list = [x.numpy() for x in inputs]
                    
                    # Process audio for Wav2Vec2
                    features = self.feature_extractor(
                        input_list,
                        sampling_rate=16000,
                        return_tensors="pt",
                        padding=True
                    )
                    features = {k: v.to(self.device) for k, v in features.items()}
                    
                    outputs = self.wav2vec_model(**features)
                    logits = outputs.logits
                    
                elif model_type == 'yamnet':
                    # Ensure input tensor has the correct shape [batch_size, 1, time_steps]
                    if inputs.dim() == 2:  # If input is [batch_size, time_steps]
                        inputs = inputs.unsqueeze(1)  # Add channel dimension
                    elif inputs.dim() == 3 and inputs.size(1) != 1:  # If input is [batch_size, channels, time_steps]
                        # Average across channels to convert to mono
                        inputs = inputs.mean(dim=1, keepdim=True)
                    
                    # Move inputs to the correct device
                    inputs = inputs.to(self.device)
                    
                    # Get predictions from YAMNet
                    logits = self.yamnet_model(inputs)
                    
                else:
                    raise ValueError(f"Unknown model type: {model_type}")
                
                # Convert logits to probabilities
                probs = torch.nn.functional.softmax(logits, dim=-1)
                
                all_probs.append(probs.cpu().numpy())
                all_labels.append(labels.cpu().numpy())
        
        return np.vstack(all_probs), np.concatenate(all_labels)

    def majority_voting(
        self,
        wav2vec_preds: np.ndarray,
        yamnet_preds: np.ndarray
    ) -> np.ndarray:
        """Combine predictions using majority voting"""
        # Stack predictions and take mode along axis 1
        stacked_preds = np.vstack([wav2vec_preds, yamnet_preds])
        return np.apply_along_axis(
            lambda x: np.bincount(x).argmax(),
            axis=0,
            arr=stacked_preds
        )

    def weighted_average(
        self,
        wav2vec_probs: np.ndarray,
        yamnet_probs: np.ndarray,
        wav2vec_weight: float = 0.6
    ) -> np.ndarray:
        """Combine predictions using weighted average of probabilities"""
        yamnet_weight = 1 - wav2vec_weight
        weighted_probs = (wav2vec_weight * wav2vec_probs + 
                         yamnet_weight * yamnet_probs)
        return np.argmax(weighted_probs, axis=1)

    def train_meta_classifier(
        self,
        wav2vec_probs: np.ndarray,
        yamnet_probs: np.ndarray,
        true_labels: np.ndarray
    ) -> None:
        """Train meta-classifier on combined probabilities"""
        # Combine features from both models
        combined_features = np.hstack([wav2vec_probs, yamnet_probs])
        
        # Initialize and train meta-classifier
        self.meta_classifier = RandomForestClassifier(
            n_estimators=100,
            random_state=42
        )
        self.meta_classifier.fit(combined_features, true_labels)

    def evaluate_ensemble(
        self,
        wav2vec_probs: np.ndarray,
        yamnet_probs: np.ndarray,
        true_labels: np.ndarray
    ) -> Dict[str, Dict[str, float]]:
        """Evaluate all ensemble methods"""
        results = {}
        best_method = None
        best_f1 = -1
        ensemble_params = {}
    
        # Get predictions for each method
        predictions = {
            'wav2vec': np.argmax(wav2vec_probs, axis=1),
            'yamnet': np.argmax(yamnet_probs, axis=1),
            'majority_voting': self.majority_voting(
                np.argmax(wav2vec_probs, axis=1),
                np.argmax(yamnet_probs, axis=1)
            ),
            'weighted_average': self.weighted_average(wav2vec_probs, yamnet_probs, wav2vec_weight=0.6)
        }
    
        ensemble_params['weighted_average'] = {'wav2vec_weight': 0.6}
    
        if self.meta_classifier is not None:
            combined_features = np.hstack([wav2vec_probs, yamnet_probs])
            predictions['meta_classifier'] = self.meta_classifier.predict(combined_features)
            ensemble_params['meta_classifier'] = {
                'type': type(self.meta_classifier).__name__,
                'params': self.meta_classifier.get_params()
            }
    
        # Calculate metrics for each method
        for method, preds in predictions.items():
            precision, recall, f1, _ = precision_recall_fscore_support(
                true_labels,
                preds,
                average='weighted'
            )
        
            results[method] = {
                'accuracy': accuracy_score(true_labels, preds),
                'precision': precision,
                'recall': recall,
                'f1': f1
            }
        
            # Track best method
            if f1 > best_f1:
                best_f1 = f1
                best_method = method
    
        # Add best method info to results
        results['best_method'] = {
            'name': best_method,
            'f1_score': best_f1,
            'parameters': ensemble_params.get(best_method, {})
        }
    
        return results

    def save_results(self, results: Dict[str, Dict[str, float]], output_dir: str) -> None:
        """Save evaluation results to JSON file"""
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, 'evaluation_results.json')
    
        with open(output_path, 'w') as f:
            json.dump(results, f, indent=4)
    
        # Print best method information
        best_method = results['best_method']
        print(f"\nBest performing method: {best_method['name']}")
        print(f"F1 Score: {best_method['f1_score']:.4f}")
        if best_method['parameters']:
            print("Parameters:")
            for param, value in best_method['parameters'].items():
                print(f"  {param}: {value}")
    
        print(f"\nResults saved to {output_path}")

    def process_audio(
        self,
        audio_path: str,
        target_sr: int = 16000
    ) -> torch.Tensor:
        """Process audio file for model input"""
        # Load and resample audio
        audio, sr = librosa.load(audio_path, sr=None)
        if sr != target_sr:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=target_sr)
        
        # Convert to tensor
        audio_tensor = torch.FloatTensor(audio)
        
        # Add batch dimension
        audio_tensor = audio_tensor.unsqueeze(0)
        
        return audio_tensor

class AudioDatasetWithMetadata(Dataset):
    """Dataset for loading preprocessed audio files with metadata"""
    def __init__(
        self,
        audio_dir: str,
        metadata_file: str,
        sample_rate: int = 16000
    ):
        self.audio_dir = audio_dir
        self.sample_rate = sample_rate
        
        # Load metadata
        self.metadata = pd.read_csv(metadata_file)
        
        # Create label mapping
        unique_labels = sorted(self.metadata['label'].unique())
        self.label_map = {label: idx for idx, label in enumerate(unique_labels)}
        
        print(f"Loaded dataset with {len(self.metadata)} samples")
        print("Label mapping:", self.label_map)
        
    def __len__(self) -> int:
        return len(self.metadata)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        row = self.metadata.iloc[idx]
        audio_path = os.path.join(self.audio_dir, row['file_name'])
        label = self.label_map[row['label']]
        
        try:
            # Load audio
            audio, _ = librosa.load(audio_path, sr=self.sample_rate)
            
            # Convert to tensor
            audio_tensor = torch.FloatTensor(audio)
            
            return audio_tensor, label
            
        except Exception as e:
            print(f"Error loading {audio_path}: {str(e)}")
            # Return a zero tensor and the label if loading fails
            return torch.zeros(self.sample_rate * 2), label  # 2 seconds of zeros
        
def cleanup_dataloader():
    torch.utils.data._utils.worker._worker_info = None

def main():
    # Set paths
    base_dir = "Split_Data"
    wav2vec_path = "model_output/best_model"
    yamnet_path = "yamnet_model_output/best_model"
    output_dir = "ensemble_output"

    # Register cleanup function
    atexit.register(cleanup_dataloader)

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    try:
        # Initialize ensemble classifier
        print("Initializing ensemble classifier...")
        ensemble = EnsembleClassifier(wav2vec_path, yamnet_path)
        
        # Create test dataset and dataloader
        print("\nCreating test data loader...")
        test_dataset = AudioDatasetWithMetadata(
            audio_dir=os.path.join(base_dir, "test"),
            metadata_file=os.path.join(base_dir, "test_metadata.csv"),
            sample_rate=16000
        )
        
        test_loader = DataLoader(
            test_dataset,
            batch_size=32,
            shuffle=False,
            num_workers=4,
            pin_memory=True
        )
        
        # Get predictions from both models
        print("\nGetting model predictions...")
        wav2vec_probs, true_labels = ensemble.get_model_predictions(
            test_loader, 'wav2vec'
        )
        yamnet_probs, _ = ensemble.get_model_predictions(
            test_loader, 'yamnet'
        )
        
        # Train meta-classifier using validation set
        print("\nCreating validation data loader for meta-classifier training...")
        val_dataset = AudioDatasetWithMetadata(
            audio_dir=os.path.join(base_dir, "val"),
            metadata_file=os.path.join(base_dir, "val_metadata.csv"),
            sample_rate=16000
        )
        
        val_loader = DataLoader(
            val_dataset,
            batch_size=32,
            shuffle=False,
            num_workers=0, # Disable workers for smaller dataset
            pin_memory=True
        )
        
        print("\nGetting validation set predictions for meta-classifier training...")
        val_wav2vec_probs, val_labels = ensemble.get_model_predictions(
            val_loader, 'wav2vec'
        )
        val_yamnet_probs, _ = ensemble.get_model_predictions(
            val_loader, 'yamnet'
        )
        
        print("\nTraining meta-classifier...")
        ensemble.train_meta_classifier(
            val_wav2vec_probs,
            val_yamnet_probs,
            val_labels
        )
        
        # Evaluate ensemble methods
        print("\nEvaluating ensemble methods...")
        results = ensemble.evaluate_ensemble(
            wav2vec_probs,
            yamnet_probs,
            true_labels
        )
        
        # Generate visualizations
        print("\nGenerating visualizations...")
        ensemble.visualizer.plot_confusion_matrix(
            confusion_matrix(true_labels, np.argmax(wav2vec_probs, axis=1)),
            method_name="Wav2Vec2",
            output_dir=output_dir
        )
        ensemble.visualizer.plot_confusion_matrix(
            confusion_matrix(true_labels, np.argmax(yamnet_probs, axis=1)),
            method_name="YAMNet",
            output_dir=output_dir
        )
        ensemble.visualizer.plot_roc_curves(
            {
                "Wav2Vec2": wav2vec_probs,
                "YAMNet": yamnet_probs
            },
            true_labels,
            output_dir
        )
        ensemble.visualizer.plot_performance_comparison(
            results,
            output_dir
        )
        ensemble.visualizer.plot_prediction_distribution(
            {
                "Wav2Vec2": np.argmax(wav2vec_probs, axis=1),
                "YAMNet": np.argmax(yamnet_probs, axis=1)
            },
            output_dir
        )
        
        # Save results
        ensemble.save_results(results, output_dir)
        
        # Save label mapping
        with open(os.path.join(output_dir, 'label_map.json'), 'w') as f:
            json.dump(test_dataset.label_map, f, indent=4)
        
        print("\nEnsemble evaluation complete! Check the output directory for visualizations.")
        
    except Exception as e:
        print(f"\nError during execution: {str(e)}")
        raise

    finally:
        with SuppressOutput():
            cleanup_dataloader()

if __name__ == "__main__":
    main()

2025-02-23 21:20:44.066401: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 21:20:44.077390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740325844.091708   21226 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740325844.095588   21226 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-23 21:20:44.109789: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Initializing ensemble classifier...
Loading Wav2Vec2 model...
Loading YAMNet model...

Creating test data loader...
Loaded dataset with 671 samples
Label mapping: {'crying': 0, 'normal': 1, 'screaming': 2}

Getting model predictions...


Getting wav2vec predictions:   0%|          | 0/21 [00:00<?, ?it/s]

Getting yamnet predictions:   0%|          | 0/21 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cacfd71a200><function _MultiProcessingDataLoaderIter.__del__ at 0x7cacfd71a200><function _MultiProcessingDataLoaderIter.__del__ at 0x7cacfd71a200><function _MultiProcessingDataLoaderIter.__del__ at 0x7cacfd71a200>


Traceback (most recent call last):

Traceback (most recent call last):
  File "/home/pratyush/miniconda3/envs/tf_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
Traceback (most recent call last):
Traceback (most recent call last):
      File "/home/pratyush/miniconda3/envs/tf_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
  File "/home/pratyush/miniconda3/envs/tf_env/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
self._shutdown_workers()    
  File "/home/pratyush/miniconda3/envs/tf_env/lib/python3.10/site-packages/to


Creating validation data loader for meta-classifier training...
Loaded dataset with 670 samples
Label mapping: {'crying': 0, 'normal': 1, 'screaming': 2}

Getting validation set predictions for meta-classifier training...


Getting wav2vec predictions:   0%|          | 0/21 [00:00<?, ?it/s]

Getting yamnet predictions:   0%|          | 0/21 [00:00<?, ?it/s]


Training meta-classifier...

Evaluating ensemble methods...

Generating visualizations...

Best performing method: meta_classifier
F1 Score: 0.9568
Parameters:
  type: RandomForestClassifier
  params: {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}

Results saved to ensemble_output/evaluation_results.json

Ensemble evaluation complete! Check the output directory for visualizations.
